In [1]:
import numpy as np
import numpy
import matplotlib.pyplot as plt
# import seaborn as sns
import keras
import tensorflow as tf
from tensorflow import keras
import numpy as np
# from tensorflow.math import confusion_matrix
import cv2
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
image_dir = 'Directory of the dataset'
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
image_files = sorted(image_files, key=lambda x: int(
    x.split('.')[0]))  # sort by numerical value
new_size = (28, 28)
X_train = []
for file in image_files:
    image_path = os.path.join(image_dir, file)
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, new_size)
    resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    X_train.append(resized_image)

In [ ]:
plt.imshow(X_train[0])

plt.imshow(X_train[350], cmap='Greys')

In [4]:
y_train = pd.read_csv("Path of label file")
y_train_nparray = np.array(y_train)

In [ ]:
numpy.shape(X_train),y_train.shape

In [ ]:
y_train_nparray.shape

In [7]:
X_train = np.array(X_train)
X_train = np.expand_dims(X_train, -1)
X_train = X_train / 255.0

In [8]:
X_train, X_validation, y_train, y_validation = train_test_split(
    X_train, y_train_nparray, test_size=0.2, random_state=2020)

In [ ]:
numpy.shape(X_train),  y_train.shape, numpy.shape(X_validation), y_validation.shape

In [10]:
# Define the model
cnn_model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, strides=(
        1, 1), padding='valid', activation='relu', input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.2),  # Add dropout layer to reduce overfitting
    keras.layers.Conv2D(filters=64, kernel_size=3, strides=(
        1, 1), padding='valid', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.2),  # Add dropout layer to reduce overfitting
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.2),  # Add dropout layer to reduce overfitting
    keras.layers.Dense(units=63, activation='softmax')
])

In [ ]:
cnn_model.summary()

In [12]:
cnn_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnn_model.fit(X_train, y_train, epochs=30, validation_split=0.1)

In [ ]:
loss, accuracy = cnn_model.evaluate(X_validation, y_validation)
print('Validation accuracy:', accuracy)

In [ ]:
Y_pred = cnn_model.predict(X_validation)

In [ ]:
print(X_validation.shape)
# first data point in X_test
plt.imshow(X_validation[0])
plt.show()

In [ ]:
input_image_path = input('path of image for prediction')

input_image = cv2.imread(input_image_path)

plt.imshow(input_image)
plt.show()

grayscale = cv2.cvtColor(input_image, cv2.COLOR_RGB2GRAY)

input_image_resize = cv2.resize(grayscale, (28, 28))

input_image_resize = input_image_resize/255

image_reshaped = np.expand_dims(input_image_resize, axis=(0, -1))

input_prediction = cnn_model.predict(image_reshaped)

input_pred_label = np.argmax(input_prediction)


def convert_to_base62(num):
    if num < 10:
        return str(num)
    elif num < 36:
        return chr(num + 55)
    elif num < 62:
        return chr(num + 61)
    else:
        return None


print('The character is predicted to be', convert_to_base62(input_pred_label))